# Pipelines
##### Pipelines (Конвейеры) - это простой способ упорядочить код предварительной обработки данных и моделирования. В частности, конвейер объединяет этапы предварительной обработки и моделирования, поэтому вы можете использовать весь пакет (Bundle) так, как если бы это был один шаг.

#### Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../datasets/melb_data.csv")

# Для примера удалим лишние столбцы
# data = data.drop(['Suburb', 'Address', 'SellerG', 'Date', 'CouncilArea'], axis=1)

# Целевые значения
y = data.Price

# Отбросим колонку целевых значений из датасета
X = data.drop(["Price"], axis=1)

X_train_full, X_test_full, y_train, y_test = train_test_split = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

# Уберем категоральные столбцы с низкой мощностью ("Мощность" означает количество уникальных значений в столбце)
categorical_cols = [cname for cname in X_train_full.columns
                    if X_train_full[cname].nunique() < 10
                    and X_train_full[cname].dtype == "object"]

# Получим все числовые столбцы
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

### Шаг 1
#### Определение этапов предварительной обработки

Подобно тому, как конвейер объединяет этапы предварительной обработки и моделирования, мы используем класс ColumnTransformer для объединения различных этапов предварительной обработки

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [4]:
# Предварительная обработка числовых данных. SimpleImputer - oдномерный imputer для заполнения пропущенных значений с помощью простых стратегий.
numerical_transformer = SimpleImputer(strategy='constant')

# Предварительная обработка категориальных данных
categorical_transformer = Pipeline(steps=[
    # Восстанавливаем пропущенные данные самым частым значением
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # Кодируем категориальные объекты в виде однозначного числового массива
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

# Предварительная обработка пакета (Bundle) для числовых и категориальных данных
preprocessor = ColumnTransformer(
    transformers=[
        # (название, трансформер, столбцы к которым применить обработку)
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols,)
    ]
)

### Шаг 2
#### Определение модели

Далее определим модель случайного леса с помощью класса RandomForestRegressor

In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=111,
    random_state=42,
)

### Шаг 3
#### Создание и оценка конвейера

Используем класс Pipeline для определения конвейера, который объединяет этапы предварительной обработки и моделирования. Следует отметить несколько важных моментов:

- С помощью конвейера мы предварительно обрабатываем обучающие данные и помещаем модель в одну строку кода
- С помощью конвейера мы передаем необработанные объекты в X_test команде predict(), и конвейер автоматически выполняет предварительную обработку объектов перед генерацией прогнозов

In [6]:
from sklearn.metrics import mean_absolute_error

In [7]:
# Объединим предварительную обработку и моделирование кода в конвейер
my_over_9000_pepline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ]
)

# Предварительная обработка обучающих данных, тренировка модели
my_over_9000_pepline.fit(X_train, y_train)

# Предварительная обработка проверочных данных, получение прогнозов
preds = my_over_9000_pepline.predict(X_test)

# Оценка модели
score = mean_absolute_error(y_test, preds)
print("MAE:", score)

MAE: 162333.39220811386
